In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm.notebook import tqdm as tqdm
import pdb

In [8]:
class LiquidNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LiquidNeuralNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.layers = nn.ModuleList([self._create_layer(input_size, hidden_size) for _ in range(num_layers)])

    def _create_layer(self, input_size, hidden_size):
        return nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_size, hidden_size)
        )

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = layer(x)
        return x

In [9]:
class ODESolver(nn.Module):
    def __init__(self, model, dt):
        super(ODESolver, self).__init__()
        self.model = model
        self.dt = dt

    def forward(self, x):
        with torch.enable_grad():
            outputs = []
            for i, layer in enumerate(self.model.layers):
                outputs.append(layer(x))
                x = outputs[-1]
        return x

    def loss(self, x, t):
        with torch.enable_grad():
            outputs = []
            for i, layer in enumerate(self.model.layers):
                outputs.append(layer(x))
                x = outputs[-1]
        return x

In [30]:
def train(model, dataset, optimizer, epochs, batch_size):
    model.train()
    total_loss = 0
    epochs_bar = tqdm(range(epochs), leave=False)
    batches_bar = tqdm(dataset, leave=False)
    for epoch in range(epochs):
        epochs_bar.update()
        for batch in dataset:
            inputs = batch
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = model.loss(inputs, outputs)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            batches_bar.set_description(f'Epoch {epoch+1}, Loss: {total_loss / len(dataset)}')

In [ ]:
train(ODESolver(LiquidNeuralNetwork(1, 100, 10), 0.01), torch.randn(1000, 1), optim.Adam, 10, 100

In [43]:
model = LiquidNeuralNetwork(1, 20, 3)
optimizer = optim.Adam(model.parameters(), lr=0.001)
dataset = torch.randn(100, 100)
train(ODESolver(model, 0.01), dataset, optimizer, 100, 100)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x100 and 1x20)

In [44]:
# Example Usage

# Define model parameters
input_size = 10
hidden_size = 20
num_layers = 3
dt = 0.01
epochs = 10
batch_size = 32

# Create a LiquidNeuralNetwork
model = LiquidNeuralNetwork(input_size, hidden_size, num_layers)

# Create an ODESolver with the model
ode_solver = ODESolver(model, dt)

# Dummy dataset (replace this with your actual dataset)
dummy_data = torch.randn((100, input_size))

# Dummy optimizer (replace this with your desired optimizer)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
train(model=ode_solver, dataset=dummy_data, optimizer=optimizer, epochs=epochs, batch_size=batch_size)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x20 and 10x20)